#### usage of yolo for getting numbers and letters only
#### lib and function def

In [1]:
pwd

'D:\\Proj_OCR'

In [2]:
import cv2
import torch
from PIL import Image
import cv2
import torch
from PIL import Image
import glob
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.image as mpimg
import os
from pyzbar.pyzbar import decode
from pyzbar import pyzbar
import cv2
import glob
from tqdm import tqdm
from barcode import EAN13
from barcode.writer import ImageWriter
import matplotlib.pyplot as plt

import cv2
import torch
from PIL import Image


%matplotlib inline

In [3]:
files = glob.glob('./Raw_Stickers2/Type_1/*.png')
print(len(files))
another_files = glob.glob('./Raw_Stickers2/Type_1/*.jpg')

for file in tqdm(another_files):
    files.append(file)
    
print(len(files))

0


100%|██████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<?, ?it/s]

36


#### Model loading

In [4]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./Models/best_numlet_labelType1.pt')
model.iou = 0.1
print(type(model))

Using cache found in C:\Users\sendr/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-12-9 torch 1.8.2+cu111 CUDA:0 (GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
Adding AutoShape... 


<class 'models.common.AutoShape'>


In [5]:
class NumberLetter:
    def __init__(self, image = None):
        self.image = image
        self.results = []
        self.letternumbers = []

In [11]:
pwd

'D:\\Proj_OCR'

In [6]:
imgs = []  # batch of images

print('--Loading Images--')
for file in tqdm(files):
    imgs.append(NumberLetter(cv2.imread(file)))

print('--Converting Image Color to RGB--')   
for i in tqdm(range (len(imgs))):
    imgs[i].image = cv2.cvtColor(imgs[i].image, cv2.COLOR_BGR2RGB)

print('Identifying crop locations...')
for i in tqdm(range (len(imgs))):
    imgs[i].results = model(np.copy(imgs[i].image))

print('Cropping...')
for i in tqdm(range (len(imgs))):
    #imgs[i].results.save()
    df = imgs[i].results.pandas().xyxy[0]
    
    if (len(df) >0):
        for j in range (len(df)):
            xmin = int(df.iloc[j].xmin)
            ymin = int(df.iloc[j].ymin)
            xmax = int(df.iloc[j].xmax)
            ymax = int(df.iloc[j].ymax)
            imgs[i].letternumbers.append(imgs[i].image[ymin:ymax,xmin:xmax])
            
#Collect all stickers from all images
print ('Collect all stickers from all images...')
letternumber_list = []

for img in tqdm(imgs):
    for letternumber in img.letternumbers:
        letternumber_list.append(letternumber)

#check before output  
print('Total letter numbers:',len(letternumber_list))
print(type(letternumber_list[0]))
print()

print('Convert color back to BGR...')
for i in tqdm (range (len(letternumber_list))):
    try:
        #sticker[i] = cv2.cvtColor(sticker[i], cv2.COLOR_BGR2RGB)
        letternumber_list[i] = letternumber_list[i][:,:,::-1]
    except:
        continue

#outputting result
print('outputting result...')
i = 0
prefix = ''
for letternumber in tqdm(letternumber_list):
    
    if (i < 10):
        prefix = '00'
    elif (i < 100):
        prefix = '0'
    elif (i>100):
        prefix = ''
        
    cv2.imwrite('./Numbers_and_Letters2/nl2a_'+prefix+str(i)+'.jpg',letternumber)
    i+=1
            

--Loading Images--


100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:01<00:00, 21.23it/s]


--Converting Image Color to RGB--


100%|█████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 427.16it/s]


Identifying crop locations...


...

100%|███████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 99600.89it/s]


Total letter numbers: 252
<class 'numpy.ndarray'>

Convert color back to BGR...


100%|████████████████████████████████████████████████████████████████████████████| 252/252 [00:00<00:00, 227892.33it/s]


outputting result...


100%|███████████████████████████████████████████████████████████████████████████████| 252/252 [00:00<00:00, 516.99it/s]
